In [1]:
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/r1.0

Cloning into 'waymo-od'...
remote: Enumerating objects: 1206, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 1206 (delta 176), reused 220 (delta 104), pack-reused 884
Receiving objects: 100% (1206/1206), 25.57 MiB | 9.57 MiB/s, done.
Resolving deltas: 100% (756/756), done.
* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/master
  remotes/origin/om2
  remotes/origin/r1.0
  remotes/origin/r1.0-tf1.15
  remotes/origin/r1.0-tf2.0
  remotes/origin/r1.2
Note: checking out 'remotes/origin/r1.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at

In [ ]:
#!pip install --upgrade pip
#!pip install waymo-open-dataset
#!pip install pandas
#!pip install matplotlib
#!pip install nltk
#!pip install pytest
#!pip install tensorflow
#!pip install --upgrade pip

In [2]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0}) # Ignore warning about 20+ figures consuming memory
import matplotlib.patches as patches

In [3]:
import nltk
nltk.download('punkt')

#import nltk
#import ssl

#try:
#    _create_unverified_https_context = ssl._create_unverified_context
#except AttributeError:
#    pass
#else:
#    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download()

[nltk_data] Downloading package punkt to /Users/wang_ja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
def sorting_data(frame_data, frame_num):
    nltk_tokens = nltk.word_tokenize(str(frame)) # Tokenize frame data
    df = pd.DataFrame() # Creating an empty dataframe
    for i in range(len(nltk_tokens)): # For i in list indexes
        if nltk_tokens[i] == "laser_labels": # Assign elements named "laser_labels" with index i
            laser_data = [[ nltk_tokens[i], nltk_tokens[i+47], nltk_tokens[i+6], nltk_tokens[i+9], nltk_tokens[i+12], \
                nltk_tokens[i+15], nltk_tokens[i+18], nltk_tokens[i+21], nltk_tokens[i+24], nltk_tokens[i+30],\
                nltk_tokens[i+33], nltk_tokens[i+36], nltk_tokens[i+39], nltk_tokens[i+43] ]]
            df = df.append(laser_data, ignore_index=True) # append the above parameters to empty df
    df.columns = ['device','iD', 'center_x', 'center_y', 'center_z', 'width', 'length', 'height', \
                'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y', 'type']; # add parameter column titles

    df = df[df['type'] != 'TYPE_PEDESTRIAN'] # ignore pedestiran data

    df[['center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y']] = df[['center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y']].astype('float64'); # turn df data into float varibales

    # Convert DataFrame to .xslx
    frame_xlsx ='frame' + str(frame_num) + '.xlsx'
    df.to_excel(frame_xlsx)

    return df

In [ ]:
def show_camera_image(camera_image, camera_labels, layout, cmap=None):
  """Show a camera image and the given camera labels"""

  ax = plt.subplot(*layout)

  # Draw the camera labels
  for camera_labels in frame.camera_labels:
    # Ignore camera labels that do not correspond to this camera
    if camera_labels.name != camera_image.name:
      continue

    # Iterate over the individual labels
    for label in camera_labels.labels:
      # Draw the object bounding box
      ax.add_patch(patches.Rectangle(
        xy=(label.box.center_x - 0.5 * label.box.length,
            label.box.center_y - 0.5 * label.box.width),
        width=label.box.length,
        height=label.box.width,
        linewidth=1,
        edgecolor='red',
        facecolor='none'))

  # Show/download camera image
  plt.imshow(tf.image.decode_jpeg(camera_image.image), cmap=cmap)
  plt.title(open_dataset.CameraName.Name.Name(camera_image.name))
  plt.grid(False)
  plt.axis('off')
  frame_img = 'frame' + str(i) + '.png'
  plt.savefig(frame_img)

In [ ]:
def find_AV_speed_dis(df1, df2, frame_num):
    sign_in1 = df1[df1['type'] == 'TYPE_SIGN'] # filter out rows with type 'TYPE_SIGN' from df1
    sign_in2 = df2[df2['type'] == 'TYPE_SIGN'] # filter out rows with type 'TYPE_SIGN' from df2

    for i in range(len(sign_in1.iloc[:, 1])): # for i in iD column in sign_in1
        for j in range(len(sign_in2.iloc[:, 1])): # for j in iD column in sign_in2
            if sign_in1.iloc[i, 1] == sign_in2.iloc[j, 1]: # if two signs have the same iD
                AV_x_travelled = sign_in1.iloc[i, 2] - sign_in2.iloc[j, 2] # sign x coordinate difference 
                AV_y_travelled = sign_in1.iloc[i, 3] - sign_in2.iloc[j, 3] # sign y coordinate difference 

                AV_x_speed = AV_x_travelled/0.1 # AV x distance travelled/0.1s between frames, unit = m/s
                AV_y_speed = AV_y_travelled/0.1 # AV y distance travelled/0.1s between frames, unit = m/s

    dis = math.sqrt(AV_x_travelled**2 + AV_y_travelled**2) # total distance travelled
    
    AV_heading_change = -math.atan(AV_y_travelled/AV_x_travelled)
    
    return dis, AV_x_speed, AV_y_speed, AV_x_travelled, AV_y_travelled, AV_heading_change

## Finding Traffic Flow

In [ ]:
def find_traffic_flow(df1, df2, frame_num, AV_cum_dist, dis, AV_x_speed, AV_y_speed, cum_heading, AV_heading_change, final_data):
    
    # sort vehicle data:
    AV_row = {'device':'laser_labels', 'iD':'av', 'center_x':0, 'center_y':0, 'center_z':0, 'width':2.03, 'length':5.18, 'height':1.78, 'heading':0, 'speed_x':AV_x_speed, 'speed_y':AV_y_speed, 'accel_x':np.NaN, 'accel_y':np.NaN, 'type':'TYPE_VEHICLE'}
    veh_in2 = df2[df2['type'] == 'TYPE_VEHICLE'] # filter df2 with only TYPE_VEHICLE
    veh_in2 = veh_in2.append(AV_row, ignore_index=True) # add AV_row to filtered df2
    for i in range(len(veh_in2)):
        car_x = veh_in2.iloc[i,2]
        car_y = veh_in2.iloc[i,3]
        r = -dis/AV_heading_change

        # filter out vehicles in other lanes, lane width = 3.7m
        if r>0: # negative heading
            if math.sqrt((car_y-r)**2+car_x**2)>r-1.85:
                if math.sqrt((car_y-r)**2+car_x**2)<r+1.85:
                    print(car_x)
        if r<0: # positive heading
            if math.sqrt((car_y-r)**2+car_x**2)<1.85-r:
                if math.sqrt((car_y-r)**2+car_x**2)>-1.85-r:
                    print(car_x)
    veh_in2 = veh_in2[abs(veh_in2['heading']) <= math.pi/4] # filter out vehicles in other heading directions
    veh_in2 = veh_in2[abs(veh_in2['center_y']) <= 3.7/2] # filter out vehicles in other lanes, US road width = 3.7m
    veh_in2 = veh_in2.sort_values(by=['center_x']) # sort data to ascending x values

    # count number of vehicles detected and detection range:
    veh_in3 = veh_in2[veh_in2['iD']!='av']
    veh_detected = veh_in3["center_x"].count()
    veh_detected_position = []
    for i in veh_in3["center_x"]:
        veh_detected_position.append(i)

    # spacing and density:
    list_1 = []
    for i in range(len(veh_in2.iloc[:, 2])-1):
        spacing = abs(veh_in2.iloc[i, 2]-veh_in2.iloc[i+1, 2])
        list_1.append(spacing)
    avg_spacing = sum(list_1)/(len(veh_in2.iloc[:, 2])-1) # unit = m
    density = 1000/avg_spacing # k = 1000/s, unit = veh/km

    # space mean speed:
    list_2 = []
    for i in range(len(veh_in2.iloc[:, 9])):
        veh_x_speed = veh_in2.iloc[i, 9]
        veh_y_speed = veh_in2.iloc[i, 10]
        veh_speed = 3.6*math.sqrt(veh_x_speed**2 + veh_y_speed**2) # unit = km/hr
        list_2.append(veh_speed)
        list_2 = [i for i in list_2 if i > 1] # change 0 velocity target
    avg_veh_speed = len(list_2)/sum(1/i for i in list_2) # veh/sum(1/velocity), unit = km/hr
    
    # headway:
    headway = 3.6*avg_spacing/avg_veh_speed # spacing/speed, unit = sec/veh

    # flow:
    flow = density*avg_veh_speed # q = k * v, unit = veh/hr

    # cumulative distance:
    dis_travelled = {'AV Cumulative Distance (m)':[dis]}
    cum_dist_df = pd.DataFrame(dis_travelled, columns = ['AV Cumulative Distance (m)'])
    AV_cum_dist = AV_cum_dist.append(cum_dist_df) 

    # AV speed:
    AV_speed = 3.6*math.sqrt(AV_x_speed**2 + AV_y_speed**2) # unit = km/hr

    # AV heading using initial heading as reference zero:
    heading_travelled = {'AV Cumulative Heading': [AV_heading_change]}
    cum_heading_df = pd.DataFrame(heading_travelled, columns = ['AV Cumulative Heading'])
    AV_cum_heading = cum_heading.append(cum_heading_df) 

    # add measurements to a df:
    measures = {'Frame': [frame_num], \
                'Time (s)': [0.1*(frame_num-1)], \
                'Flow (veh/h)':[flow], \
                'Density (veh/km)':[density], \
                'Spacing (m)':[avg_spacing], \
                'Headway (s)':[headway], \
                'Space mean speed (km/hr)':[avg_veh_speed], \
                'AV distance travelled (m)':[AV_cum_dist.sum().sum()], \
                'AV heading (deg)':[(cum_heading.sum().sum())*180/3.1415926536], \
                'AV speed (km/hr)':[AV_speed], \
                'Number of cars detected':[veh_detected], \
                'Cars detected':[veh_detected_position]}
    traffic_measure = pd.DataFrame(measures, columns = ['Frame', \
                                                        'Time (s)', \
                                                        'Flow (veh/h)', \
                                                        'Density (veh/km)', \
                                                        'Spacing (m)', \
                                                        'Headway (s)', \
                                                        'Space mean speed (km/hr)', \
                                                        'AV distance travelled (m)', \
                                                        'AV heading (deg)', \
                                                        'AV speed (km/hr)', \
                                                        'Number of cars detected', \
                                                        'Cars detected'])
    final_data = final_data.append(traffic_measure, ignore_index = True)

    return final_data, AV_cum_dist, AV_cum_heading

In [18]:
# Parameters
j = 2 # Start frame; min = 1, change for specific frames
k = 200 # End frame; max = 200

i = 0 # Frame count; initialise as 0
i2 = 1 # Frame pair count; initialise as 1

# create empty dataframes
final_data_append = pd.DataFrame()

FILENAME = '/Users/wang_ja/Desktop/waymo/training_training_0002/segment-11392401368700458296_1086_429_1106_429_with_camera_labels.tfrecord'
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')

for data in dataset:
    i += 1 # Count number of frames
    if i<=j-1:
        continue
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))

    for index, image in enumerate(frame.images):
        if i2%2 != 0: # For 1st frame in pair (where index is odd), generate DF
            df1 = sorting_data(frame, i)
        elif i2%2 == 0: # For 2nd frame in pair (where index is even), generate DF
            df2 = sorting_data(frame, i)
            if (index % 5) == 0: # if there are five images
                final_data_append = find_traffic_flow(df1, df2, i, final_data_append)
                final_data_xlsx ='final_data' + str(i) + '.xlsx'
                final_data_append.to_excel(final_data_xlsx, sheet_name='Raw')
                #show_AV_position(AV_x_travelled, AV_y_travelled, i)
            else:
                pass
            df1=df2 # Assign 1st frame as 2nd frame
            i2 += 1

    i2 += 1

    if i==k: 
        break